In [1]:
import time
import shutil
import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)

In [2]:
import os
print(os.getcwd())  #当前路径

#更改当前路径
os.chdir('D:/file/o/队长/队长评分/分队/')
os.getcwd()

D:\code\jupyter\队长


'D:\\file\\o\\队长\\队长评分\\分队'

## 读取原数据

In [73]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import pandas as pd
import numpy as np

#存量小队
# df_all = pd.read_excel("存量小队.xlsx")
df_all = pd.read_excel('data.xlsx', 'all')
df_driver = pd.read_excel('data.xlsx', 'driver')
df_group = pd.read_excel('data.xlsx', 'group')
#公海
df_sea = pd.read_excel("公海司机.xlsx")

'\nSpyder Editor\n\nThis is a temporary script file.\n'

In [92]:
df_sea.shape # 公海
df_all.shape 
df_driver.shape
df_group.shape

(16347, 4)

(50185, 34)

(50185, 15)

(1532, 27)

In [75]:
df_group.dtypes

区域        object
城市id       int64
城市        object
小队ID       int64
队长id       int64
队长等级      object
应带队人数      int64
小队活跃区县    object
a          int64
b          int64
c          int64
d          int64
a0         int64
b0         int64
c0         int64
d0         int64
待补充a人数     int64
待补充b人数     int64
待补充c人数     int64
待补充d人数     int64
总待补充人数     int64
待剔除a人数     int64
待剔除b人数     int64
待剔除c人数     int64
总待剔除人数     int64
是否补充完      int64
是否剔除完      int64
dtype: object

In [93]:
# df_all = df_all.assign(是否补充完=0, 是否剔除完=0)
'''
df_all[['a', 'b', 'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',
       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',
       '是否剔除完']] = df_all[['a', 'b', 'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',
       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',
       '是否剔除完']].astype('int64')
'''
# 打乱df_all
from sklearn.utils import shuffle  
df_all = shuffle(df_all)  
pd.set_option("display.max_columns",None)  # 可以看到全部列
df_all.head()

"\ndf_all[['a', 'b', 'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',\n       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',\n       '是否剔除完']] = df_all[['a', 'b', 'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',\n       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',\n       '是否剔除完']].astype('int64')\n"

,区域,城市id,城市,小队ID,队长id,队长等级,应带队人数,小队活跃区县,队员id,队员等级,队员活跃区县,A类区县内a,A类区县外b,非A类区县内c,非A类区县外d,a,b,c,d,a0,b0,c0,d0,待补充a人数,待补充b人数,待补充c人数,待补充d人数,总待补充人数,待剔除a人数,待剔除b人数,待剔除c人数,总待剔除人数,是否补充完,是否剔除完
16630,华北区域,1,北京市,313776,580542174467169,B,40,朝阳区,580544721359287,B,朝阳区,0,0,1,0,4,12,16,0,11,16,12,0,7,4,0,0,11,0,0,4,4,0,0
28970,华北区域,1,北京市,183310,566050733238558,A,50,朝阳区,580543429205034,B+,朝阳区,0,0,1,0,12,17,9,0,14,20,15,0,2,3,6,0,11,0,0,0,0,0,0
3197,华北区域,1,北京市,285398,565296256781257,B,40,朝阳区,580542442521949,A,通州区,0,1,0,0,23,15,18,0,11,16,12,0,0,1,0,0,1,12,0,6,18,0,0
42492,华北区域,1,北京市,530260,563683104333824,A,50,朝阳区,563283142975492,B,朝阳区,0,0,1,0,8,6,16,0,14,20,15,0,6,14,0,0,20,0,0,1,1,0,0
18627,华北区域,1,北京市,260412,567950017091106,A,50,朝阳区,565571044574407,A,丰台区,0,1,0,0,18,7,0,0,14,20,15,0,0,13,15,0,28,4,0,0,4,0,0


In [124]:
df_driver.columns

Index(['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', '队员id',
       '队员等级', '队员活跃区县', 'A类区县内a', 'A类区县外b', '非A类区县内c', '非A类区县外d', '被补充',
       '被剔除'],
      dtype='object')

In [126]:
df_sea = shuffle(df_sea.assign(被补充=0, 被剔除=0))
df_driver = shuffle(df_driver.assign(被补充=0, 被剔除=0))
df_group = df_group.assign(是否补充完=0, 是否剔除完=0)
df_sea.head()
df_driver.head()
df_group.head()

,队员id,队员等级,队员活跃区县,重新定级,被补充,被剔除
5120,565268126241919,B+,通州区,0,0,0
9003,563520613056512,B,房山区,0,0,0
6801,565281170467911,B,丰台区,0,0,0
9425,563884094324738,B,顺义区,0,0,0
4149,580543330992911,B+,朝阳区,0,0,0


,区域,城市id,城市,小队ID,队长id,队长等级,应带队人数,小队活跃区县,队员id,队员等级,队员活跃区县,A类区县内a,A类区县外b,非A类区县内c,非A类区县外d,被补充,被剔除
13376,华北区域,1,北京市,450696,563277019422721,A,50,朝阳区,565712478740724,B+,朝阳区,0,0,1,0,0,0
11164,华北区域,1,北京市,448314,580544721308978,B,40,朝阳区,563147667931139,B,朝阳区,0,0,1,0,0,0
37617,华北区域,1,北京市,159288,564111966806016,B,40,朝阳区,564111966806016,A,大兴区,0,1,0,0,0,0
7169,华北区域,1,北京市,186228,563492307927040,B,40,朝阳区,580542453635604,B+,朝阳区,0,0,1,0,0,0
19410,华北区域,1,北京市,48766,580543411779316,B,40,朝阳区,580542299622737,A,西城区,0,1,0,0,0,0


,区域,城市id,城市,小队ID,队长id,队长等级,应带队人数,小队活跃区县,a,b,c,d,a0,b0,c0,d0,待补充a人数,待补充b人数,待补充c人数,待补充d人数,总待补充人数,待剔除a人数,待剔除b人数,待剔除c人数,总待剔除人数,是否补充完,是否剔除完
0,华北区域,1,北京市,51092,563865054814208,C,30,朝阳区,13,12,0,0,8,12,9,0,0,0,9,0,9,5,0,0,5,0,0
1,华北区域,1,北京市,129076,567950018373800,C,30,朝阳区,14,10,1,0,8,12,9,0,0,2,8,0,10,6,0,0,6,0,0
2,华北区域,1,北京市,346956,563414262550528,B,40,朝阳区,13,11,0,0,11,16,12,0,0,5,12,0,17,2,0,0,2,0,0
3,华北区域,1,北京市,301930,580542246038446,B,40,朝阳区,15,11,15,0,11,16,12,0,0,5,0,0,5,4,0,3,7,0,0
4,华北区域,1,北京市,40572,565098320759301,C,30,朝阳区,15,10,2,0,8,12,9,0,0,2,7,0,9,7,0,0,7,0,0


In [83]:
# 隐藏代码
'''
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle Code</button>""", raw=True)
'''
# 生成司机信息表和小队信息表
'''
# 存储司机信息的表
df_driver = df_all[['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', '队员id',
       '队员等级', '队员活跃区县', 'A类区县内a', 'A类区县外b', '非A类区县内c', '非A类区县外d']]
df_driver.shape
# 存储小队信息的表
df_group = df_all[['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', 'a', 'b',
       'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',
       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',
       '是否剔除完']].drop_duplicates('小队ID')

df_group.shape        

# df_group[['待补充a人数', '待剔除a人数']] = df_group[['待补充a人数', '待剔除a人数']].apply(df_compute, args=(df_group['a'], df_group['a0'], 'a'))
df_group['待补充a人数'] = df_group.apply(lambda x: max(x['a0'] - x['a'], 0), axis=1)
df_group['待剔除a人数'] = df_group.apply(lambda x: max(x['a'] - x['a0'], 0), axis=1)
df_group['待补充b人数'] = df_group.apply(lambda x: max(x['b0'] - x['b'], 0), axis=1)
df_group['待剔除b人数'] = df_group.apply(lambda x: max(x['b'] - x['b0'], 0), axis=1)
df_group['待补充c人数'] = df_group.apply(lambda x: max(x['c0'] - x['c'], 0), axis=1)
df_group['待剔除c人数'] = df_group.apply(lambda x: max(x['c'] - x['c0'], 0), axis=1)
df_group['总待补充人数'] = df_group[['待补充a人数', '待补充b人数', '待补充c人数']].apply(np.sum, axis=1)
df_group['总待剔除人数'] = df_group[[ '待剔除a人数', '待剔除b人数', '待剔除c人数']].apply(np.sum, axis=1)
df_group.head()
'''

# 生成新的df_all
'''
df_all = pd.merge(left=df_driver, right=df_group, how='left')
df_all.head()


writer = pd.ExcelWriter('data.xlsx')
df_all.to_excel(writer,'all',index=False)
df_driver.to_excel(writer,'driver',index=False)
df_group.to_excel(writer,'group',index=False)
writer.save()
'''

'\nfrom IPython.display import display_html\ndisplay_html("""<button onclick="$(\'.input, .prompt, .output_stderr, .output_error\').toggle();">Toggle Code</button>""", raw=True)\n'

"\n# 存储司机信息的表\ndf_driver = df_all[['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', '队员id',\n       '队员等级', '队员活跃区县', 'A类区县内a', 'A类区县外b', '非A类区县内c', '非A类区县外d']]\ndf_driver.shape\n# 存储小队信息的表\ndf_group = df_all[['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', 'a', 'b',\n       'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',\n       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',\n       '是否剔除完']].drop_duplicates('小队ID')\n\ndf_group.shape        \n\n# df_group[['待补充a人数', '待剔除a人数']] = df_group[['待补充a人数', '待剔除a人数']].apply(df_compute, args=(df_group['a'], df_group['a0'], 'a'))\ndf_group['待补充a人数'] = df_group.apply(lambda x: max(x['a0'] - x['a'], 0), axis=1)\ndf_group['待剔除a人数'] = df_group.apply(lambda x: max(x['a'] - x['a0'], 0), axis=1)\ndf_group['待补充b人数'] = df_group.apply(lambda x: max(x['b0'] - x['b'], 0), axis=1)\ndf_group['待剔除b人数'] = df_group.apply(lambda x: max(x['b'] - x['b0'], 0), axis=1)\ndf_group['待补充c人数'] = df_group.a

"\ndf_all = pd.merge(left=df_driver, right=df_group, how='left')\ndf_all.head()\n\n\nwriter = pd.ExcelWriter('data.xlsx')\ndf_all.to_excel(writer,'all',index=False)\ndf_driver.to_excel(writer,'driver',index=False)\ndf_group.to_excel(writer,'group',index=False)\nwriter.save()\n"

In [114]:
# 小队可能存在四种情况
# 司机
df_all.query('总待补充人数>0 and 总待剔除人数==0').shape  # 5136
df_all.query('总待补充人数==0 and 总待剔除人数==0').shape  # 0
df_all.query('总待补充人数>0 and 总待剔除人数>0').shape    # 32053
df_all.query('总待补充人数==0 and 总待剔除人数>0').shape   # 12996
# 小队
df_group.query('总待补充人数>0 and 总待剔除人数==0').shape  # 322
df_group.query('总待补充人数==0 and 总待剔除人数==0').shape  # 0
df_group.query('总待补充人数>0 and 总待剔除人数>0').shape    # 966
df_group.query('总待补充人数==0 and 总待剔除人数>0').shape   # 244

(3571, 34)

(0, 34)

(33618, 34)

(12996, 34)

(322, 27)

(0, 27)

(966, 27)

(244, 27)

In [115]:
# 待补充和待剔除人数大于0的小队的数量
322 + 966  # 待补充 1288
966 + 244  # 待剔除 1210

1288

1210

In [116]:
# 不需要(补充和剔除)的司机的数量 
df_all.shape  # 司机总数：50185人
df_all.query('(a==a0 and A类区县内a==1) | (b==b0 and A类区县外b==1) | (c==c0 and 非A类区县内c==1)').shape  # 2232人

(50185, 34)

(2232, 34)

In [117]:
# 补充完的司机表
df_bc_ok = pd.DataFrame()
# 没有补充完的司机表
df_bc_nok = pd.DataFrame()
# 剔除了的司机表
df_tc_table = pd.DataFrame()

In [118]:
#待补充的小队
# df2 = df1.query('总待补充人数>=1')
df_bc_group = df_group.query('总待补充人数>=1')
#根据总待补充人数升序排列
df_bc_group.sort_values(by="总待补充人数",ascending=True, inplace=True)

df_bc_group.shape  # 待补充小队里，小队表

<ipython-input-118-37dfc8c1d820>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bc_group.sort_values(by="总待补充人数",ascending=True, inplace=True)


(1288, 27)

In [122]:
#待剔除小队
df_tc_group = df_group.query('总待剔除人数>=1')
#根据总剔除人数降序排列
df_tc_group.sort_values(by="总待剔除人数",ascending=False, inplace=True)

'''
frames = []
for i in range(df_tc_group.shape[0]):
    group = df_tc_group.iloc[i]
    group_id = group['小队ID']
    
    if group['待补充a人数'] > 0:
        frames.append(df_driver.query('小队ID == @group_id and A类区县内a == 1'))
    if group['待补充b人数'] > 0:
        frames.append(df_driver.query('小队ID == @group_id and A类区县外b == 1'))
    if group['待补充c人数'] > 0:
        frames.append(df_driver.query('小队ID == @group_id and 非A类区县内c == 1'))
        '''
        
df_tc = df_all.query('(待剔除a人数>0 and A类区县内a==1) | (待剔除b人数>0 and A类区县外b==1) | (待剔除c人数>0 and 非A类区县内c==1)')[df_driver.columns[:-2]]
df_tc.shape  # 待剔除小队里，待剔除的司机表
df_tc_group.shape  # 待剔除小队里，小队表

<ipython-input-122-8948697bd377>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tc_group.sort_values(by="总待剔除人数",ascending=False, inplace=True)


"\nframes = []\nfor i in range(df_tc_group.shape[0]):\n    group = df_tc_group.iloc[i]\n    group_id = group['小队ID']\n    \n    if group['待补充a人数'] > 0:\n        frames.append(df_driver.query('小队ID == @group_id and A类区县内a == 1'))\n    if group['待补充b人数'] > 0:\n        frames.append(df_driver.query('小队ID == @group_id and A类区县外b == 1'))\n    if group['待补充c人数'] > 0:\n        frames.append(df_driver.query('小队ID == @group_id and 非A类区县内c == 1'))\n        "

(34917, 15)

(1210, 27)

In [123]:
# 剔除数为0的小队（这里全为待补充的小队），司机表
df_tc0 = df_all.query('总待剔除人数 == 0')[df_driver.columns[:2]]
df_tc0.shape

# 小队表
df_tc0_group = df_group.query('总待剔除人数 == 0')
df_tc0_group.shape

(3571, 2)

(322, 27)

## 补充小队

In [106]:
group = df_bc_group.query('小队ID == 183310')
# group = df_bc_group.iloc[0]

#待补充人数最少的小队对应的待补充a,b,c人数和小队活跃区县
group_id = group["小队ID"]
num_a = group["待补充a人数"].iloc[0]
num_b = group["待补充b人数"].iloc[0]
num_c = group["待补充c人数"].iloc[0]
city = group["小队活跃区县"].iloc[0]
    
group

,区域,城市id,城市,小队ID,队长id,队长等级,应带队人数,小队活跃区县,a,b,c,d,a0,b0,c0,d0,待补充a人数,待补充b人数,待补充c人数,待补充d人数,总待补充人数,待剔除a人数,待剔除b人数,待剔除c人数,总待剔除人数,是否补充完,是否剔除完
797,华北区域,1,北京市,183310,566050733238558,A,50,朝阳区,12,17,9,0,14,20,15,0,2,3,6,0,11,0,0,0,0,0,0


In [163]:
# 记录小队补充情况
log_a = []
log_b = []
log_c = []

In [164]:
# 补充完的小队加入列表
a_lt = []  # 补充完的小队id加入列表
b_lt = []  
c_lt = []  

## test-先补充一个小队内的c类司机

In [29]:
df_all.columns

Index(['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', '队员id',
       '队员等级', '队员活跃区县', 'A类区县内a', 'A类区县外b', '非A类区县内c', '非A类区县外d', 'a', 'b',
       'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',
       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',
       '是否剔除完'],
      dtype='object')

In [217]:
# 补充一个小队的司机
def bc(driver, df_sea, num, abc):
    global group_id, df_tc_table
    
    if abc == 'a':
        df_sea_target = df_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机  # 按顺序取，这样可以按指定顺序取出待剔除小队的司机
        
        df_tc_table = pd.concat([df_tc_table, sea])
        sea = sea.assign(A类区县内a=[1]*len(sea) )  # 取出的司机插入新的列
        sea
        log_a.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_a.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
        
        driver_new.loc[:, '待补充a人数'] = driver_new.loc[:, '待补充a人数'] - df_sea_target.shape[0]   # 更新类别的待补充人数
        if driver_new.loc[:, '待补充a人数'] == 0:
            driver_new.loc[:, '是否补充完'] = 1
        
    elif abc == 'b':
        df_sea_target = df_sea.query('队员活跃区县 != @city and 队员等级 == "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机
        df_tc_table = pd.concat([df_tc_table, sea])
        sea = sea.assign(A类区县外b=[1]*len(sea) )  # 取出的司机插入新的列
        log_b.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_b.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
        
        driver_new.loc[:, '待补充b人数'] = driver_new.loc[:, '待补充b人数'] - df_sea_target.shape[0]   # 更新类别的待补充人数
        if driver_new.loc[:, '待补充b人数'] == 0:
            driver_new.loc[:, '是否补充完'] = 1
        
    elif abc == 'c':
        df_sea_target = df_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机
        df_tc_table = pd.concat([df_tc_table, sea])  # 将要剔除的司机的原始信息存储下来
        # 更改司机信息，补充到待补充的司机表中
        sea = sea.assign(非A类区县内c=[1]*len(sea) )  # 取出的司机插入新的列
        log_c.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_c.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
        
        driver_new.loc[:, '待补充c人数'] = driver_new.loc[:, '待补充c人数'] - df_sea_target.shape[0]   # 更新类别的待补充人数
        if driver_new.loc[:, '待补充c人数'] == 0:
            driver_new.loc[:, '是否补充完'] = 1
       
    
    
    # 加入补充的司机
    driver_new = pd.concat([driver, sea])
    driver_new = driver_new.assign(组队后小队ID=[group_id]*len(driver_new))
    
  
    return driver_new, df_sea

In [ ]:
# 补充一个小队的司机
## 谢天谢地，不需要考虑这种情况了.....
'''
def bc_from_bc(driver, df_tc0, num, abc): 
    # 补充的司机来自于另一个小队待补充的司机
    global group_id, df_tc_table
    
    if abc == 'a':
        df_sea_target = df_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机  # 按顺序取，这样可以按指定顺序取出待剔除小队的司机
        df_tc_table = pd.concat([df_tc_table, sea])
        sea = sea.assign(A类区县内a=[1]*len(sea) )  # 取出的司机插入新的列
        log_a.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_a.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
        
    elif abc == 'b':
        df_sea_target = df_sea.query('队员活跃区县 != @city and 队员等级 == "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机
        df_tc_table = pd.concat([df_tc_table, sea])
        sea = sea.assign(A类区县外b=[1]*len(sea) )  # 取出的司机插入新的列
        log_b.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_b.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
        
    elif abc == 'c':
        df_sea_target = df_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
        
        sea = df_sea_target[:num]  # 从公海取出需要的司机
        df_tc_table = pd.concat([df_tc_table, sea])  # 将要剔除的司机的原始信息存储下来
        # 更改司机信息，补充到待补充的司机表中
        sea = sea.assign(非A类区县内c=[1]*len(sea) )  # 取出的司机插入新的列
        log_c.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_c.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
       
    
    
    # 加入补充的司机
    driver_new = pd.concat([driver, sea])
    driver_new = driver_new.assign(组队后小队id=[group_id]*len(driver_new))
    # driver_new.loc[:, '是否补充完'] = 1
  
    return driver_new, df_sea
    '''

In [213]:
query_mate = '队员活跃区县 == @city and 队员等级 != "A"'
query_abc = '小队ID == @group_id and 非A类区县内c == 1'

In [216]:
# 某个小队--公海里匹配c类司机
driver_c = df_all.query(query_abc)
num_c = group["待补充c人数"].iloc[0]

df_sea_target = df_sea.query(query_mate)
num_target = num_c - df_sea_target.shape[0] 
log_c.append('{}: 需要补充{}名, 【公海里】找到{}名'.format(group_id, num_c, df_sea_target.shape[0]))
if num_target <= 0:  # 如果在公海里能找到人
    driver_c_new, df_sea = bc(driver_c, df_sea, num_c, 'c')
    c_lt.append(group_id)
    log_c.append('公海里能补充完，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))

elif num_target > 0 :
    ## 1、先补充公海里能补充的人
    df_sea_target = df_sea.query(query_mate)
    driver_c_new, df_sea = bc(driver_c, df_sea, df_sea_target.shape[0], 'c')
    
    num_c = num_target 
    log_c.append('公海里不能补充完-从公海里补充，{0}: 原有c类司机{1}名, 补充c类司机后:{2}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))  
    
    ## 2、然后从剔除人数大于0的小队补充
    df_tc_sea_target = df_tc_sea.query(query_mate)
    num_target = num_c - df_tc_sea_target.shape[0]  # num_target是 待补充人数num_c - 能够补充的人数df_sea_target.shape[0]
    log_c.append('{}: 需要补充{}名, 【待剔除人数大于0的小队】里找到{}名'.format(group_id, num_c, df_tc_sea_target.shape[0]))
    
    
    if num_target <= 0:  # 从待剔除司机里能补充完
    # 还没有检测
        driver_c_new, df_tc_sea = bc(driver_c_new, df_tc_sea,  num_c, 'c')
         c_lt.append(group_id)
        log_c.append('公海里不能补充完 -> 小队里能补充完，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))
     
    
    ## 3、仍然补充不完，则从剔除人数=0的小队补充    
    elif num_target > 0:
        
        ### 1、先补充剔除人数大于0的小队里能补充的人
        driver_c_new, df_tc_sea = bc(driver_c_new, df_tc_sea_target, df_sea_target.shape[0], 'c')
        num_c = num_target
        log_c.append('公海里不能补充完->小队里不能补完-从小队里补充，{}: 补充c类司机后:{}名'.format(group_id, driver_c_new.shape[0]))  

         # 然后跳过这个小队
        driver_new.loc[:, '是否补充完'] = 2  # 2表示补充结束，但没有补充完
        df_bc_nok = pd.concat([df_bc_nok, driver_c_new])
        log_c.append('公海里不能补充完 -> 小队里不能补充完 -> 小队0不能补充完-从小队0补充 -> 跳过这个小队，{}: 跳过该小队，有{}人未补充'.format(group_id, num_c))
          
        '''
        ### 2、然后从剔除人数等于0的小队里补充人
        df_tc0_sea_target = df_tc0.query(query_mate)
        num_target = num_c - df_tc0_sea_target.shape[0]
        log_c.append('{}: 需要补充{}名, 【待剔除人数为0的小队】里找到{}名'.format(group_id, num_c, df_tc0_sea_target.shape[0]))
        
        if num_target <= 0 : # 能补充完
            driver_c_new, df_tc0_sea = bc(driver_c_new, df_tc0_sea,  num_c, 'c')
            c_lt.append(group_id)
            log_c.append('公海里不能补充完 -> 小队里不能补充完 -> 小队0能补充完，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))
        
        elif num_target > 0:
            # 1、补充能补充的人
            driver_c_new, df_tc0_sea = bc(driver_c_new, df_tc0_sea, df_tc0_sea_targe.shape[0], 'c')
            num_c = num_target
            log_c.append('公海里不能补充完 -> 小队里不能补充完 -> 小队0不能补充完-从小队0补充，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))
            
            # 然后跳过这个小队
            driver_new.loc[:, '是否补充完'] = 2  # 2表示补充结束，但没有补充完
            df_bc_nok = pd.concat([df_bc_nok, driver_c_new])
            log_c.append('公海里不能补充完 -> 小队里不能补充完 -> 小队0不能补充完-从小队0补充 -> 跳过这个小队，{}: 跳过该小队，有{}人未补充'.format(group_id, num_c))
           '''

    
df_bc_ok = pd.concat([df_bc_ok, driver_c_new])

# log_c.clear()
log_c

NameError: name 'df_tc0_sea' is not defined

In [220]:
log_c

['312042: 需要补充8名, 公海里找到7名',
 '312042: 补充前公海有10人',
 '312042: 补充后公海有3人',
 '公海里不能补充完-从公海里补充，312042: 原有c类司机7名, 补充c类司机后:14名',
 '312042: 需要补充1名, 待剔除司机里找到0名',
 '312042: 需要补充8名, 【公海里】找到0名',
 '312042: 补充前公海有3人',
 '312042: 补充后公海有3人',
 '公海里不能补充完-从公海里补充，312042: 原有c类司机7名, 补充c类司机后:7名',
 '312042: 需要补充8名, 【待剔除人数大于0的小队】里找到0名',
 '312042: 补充前公海有0人',
 '312042: 补充后公海有0人',
 '公海里不能补充完->小队里不能补完-从小队里补充，312042: 补充c类司机后:7名',
 '312042: 需要补充8名, 【待剔除人数为0的小队】里找到922名']

In [206]:
t = df_tc0
t.columns

Index(['区域', '城市id', '城市', '小队ID', '队长id', '队长等级', '应带队人数', '小队活跃区县', '队员id',
       '队员等级', '队员活跃区县', 'A类区县内a', 'A类区县外b', '非A类区县内c', '非A类区县外d', 'a', 'b',
       'c', 'd', 'a0', 'b0', 'c0', 'd0', '待补充a人数', '待补充b人数', '待补充c人数',
       '待补充d人数', '总待补充人数', '待剔除a人数', '待剔除b人数', '待剔除c人数', '总待剔除人数', '是否补充完',
       '是否剔除完'],
      dtype='object')

In [201]:
df_sea = df_sea2[:5].append(df_sea2[-5:])
df_sea.shape

(10, 6)

In [202]:
log_c.clear()

In [204]:
driver_c_new1

,区域,城市id,城市,小队ID,队长id,队长等级,应带队人数,小队活跃区县,队员id,队员等级,队员活跃区县,A类区县内a,A类区县外b,非A类区县内c,非A类区县外d,a,b,c,d,a0,b0,c0,d0,待补充a人数,待补充b人数,待补充c人数,待补充d人数,总待补充人数,待剔除a人数,待剔除b人数,待剔除c人数,总待剔除人数,是否补充完,是否剔除完,重新定级,组队后小队id
29049,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,580544635091342,B,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29647,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,564730013950763,B,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29020,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,580542339672631,B,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29584,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,563790891130882,C,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29678,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,563456573964290,B+,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29676,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,563216107245569,B,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
29084,华北区域,1.0,北京市,312042,5.805422e+14,A,50.0,朝阳区,563079118065668,B,朝阳区,0.0,0.0,1,0.0,21.0,24.0,7.0,0.0,14.0,20.0,15.0,0.0,0.0,0.0,1,0.0,8.0,6.0,4.0,0.0,10.0,0.0,0.0,NaN,312042
2,NaN,NaN,NaN,312042,NaN,NaN,NaN,NaN,580546275940025,B+,朝阳区,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,312042
4,NaN,NaN,NaN,312042,NaN,NaN,NaN,NaN,566101595719220,B+,朝阳区,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,312042
24,NaN,NaN,NaN,312042,NaN,NaN,NaN,NaN,563168282411010,B+,朝阳区,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,312042


In [106]:
df_tc_sea.shape

(26148, 34)

In [95]:
df_tc_sea.shape

(26148, 34)

In [107]:
df_tc_sea_target.shape[0]

0

In [71]:
#公海里匹配c类司机
num_c = group["待补充c人数"].value
driver_c = df_all.query('小队ID == @group_id and 非A类区县内c == 1')

df_sea_target = df_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
num_target = num_c - df_sea_target.shape[0] 
log_c.append('{}: 需要补充{}名, 公海里找到{}名'.format(group_id, num_c, df_sea_target.shape[0]))
if num_target <= 0:  # 如果在公海里能找到人
    
    sea_c = df_sea_target.sample(num_c)  # 从公海取出需要的司机
    sea_c = sea_c.assign( 小队ID=[group_id]*len(sea_c), 非A类区县内c=[1]*len(sea_c) )  # 取出的司机插入新的列
    log_c.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
    df_sea = pd.concat([df_sea, sea_c], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
    log_c.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
    
    # 加入补充的司机
    driver_c_new = pd.concat([driver_c, sea_c])
    num_c = max(num_c - len(sea_c), 0)        # 更改补充人数 num_c  # 取max便于测试用
    driver_c_new.loc[:, '待补充c人数'] = num_c   # 更新类别的待补充人数
    driver_c_new.loc[:, '是否补充完'] = 1
    c_lt.append(group_id)
    
    log_c.append('公海里能补充完，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))

elif num_target > 0 :
    ## 1、先补充公海里能补充的人
    sea_c = df_sea_target  # 从公海取出需要的司机
    sea_c = sea_c.assign( 小队ID=[group_id]*len(sea_c), 非A类区县内c=[1]*len(sea_c) )  # 取出的司机插入新的列
    log_c.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    df_sea = pd.concat([df_sea, sea_c], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
    log_c.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
    
    # 加入补充的司机
    driver_c_new = pd.concat([driver_c, sea_c])
    num_c1 = max(num_c - len(sea_c), 0)        # 从公海补充之后，更改补充人数 num_c1  # 取max便于测试用
    driver_c_new.loc[:, '待补充c人数'] = num_c   # 更新待补充人数
    
    log_c.append('公海里不能补充完-从公海里补充，{0}: 原有c类司机{1}名, 需要补充c类司机{2}名, 补充c类司机后:{3}名'.format(group_id, driver_c.shape[0], num_c, driver_c_new.shape[0]))  
    
    ## 2、然后从剔除人数大于0的小队补充
    df_tc_sea_target = df_tc_sea.query('队员活跃区县 == @city and 队员等级 != "A"')
    num_target2 = num_c1 - df_tc_sea_target[0]
    log_c.append('{}: 需要补充{}名, 待剔除司机里找到{}名'.format(group_id, num_c1, df_tc_sea_target.shape[0]))
    
    if num_target2 <= 0:  # 从待剔除司机里能补充完
         sea_c = df_sea_target.sample(num_c)  # 从公海取出需要的司机
        sea_c = sea_c.assign( 小队ID=[group_id]*len(sea_c), 非A类区县内c=[1]*len(sea_c) )  # 取出的司机插入新的列
        log_c.append('{}: 补充前公海有{}人'.format(group_id, df_sea.shape[0]))
    
        df_sea = pd.concat([df_sea, sea_c], axis=0).drop_duplicates(subset=['队员id'], keep=False)  # 删除公海里取的司机
        log_c.append('{}: 补充后公海有{}人'.format(group_id, df_sea.shape[0]))
    
        # 加入补充的司机
        driver_c_new = pd.concat([driver_c, sea_c])
        num_c = max(num_c - len(sea_c), 0)        # 更改补充人数 num_c  # 取max便于测试用
        driver_c_new.loc[:, '待补充c人数'] = num_c   # 更新类别的待补充人数
        driver_c_new.loc[:, '是否补充完'] = 1
        c_lt.append(group_id)
    
    log_c.append('公海里能补充完，{}: 现有c类司机{}名, 补充c类司机后:{}名'.format(group_id, driver_c.shape[0], driver_c_new.shape[0]))

        
    
    ## 3、仍然补充不完，则从剔除人数=0的小队补充
    
        
    #else:
    print('暂不处理')
        # 先不处理复杂的情况，简单处理一下
        # bc_from_tc()
        # 去剔除的人里找
'''
            # if 剔除的人里能找到
            # else 去不需要剔除的人里找
    
'''
    # 最后生成小队内新的司机表  driver_c
df_bc_ok = pd.concat([df_bc_ok, driver_c_new])

# log_c.clear()
log_c

'\n            # if 剔除的人里能找到\n            # else 去不需要剔除的人里找\n    \n'

In [ ]:
    #待补充人数最少的小队表
    '''
    df7 = df1[df1['小队ID'].isin([index])]
    for i in dfa1.shape(0):
        if (dfa1[i-1:i]["队员活跃区县"] == group["小队活跃区县"] and dfa1[i-1:i]["队员等级"]=="A"):
            dfa1[i-1:i]["重新定级"]=1
        elif (dfa1[i-1:i]["队员活跃区县"]!=group["小队活跃区县"] and dfa1[i-1:i]["队员等级"]=="A"):
            dfa1[i-1,i]["重新定级"]=2
        elif (dfa1[i-1:i]["队员活跃区县"]==group["小队活跃区县"] and dfa1[i-1:i]["队员等级"]!="A"):
            dfa1[i-1:i]["重新定级"]=3
        else:
            dfa1[i-1:i]["重新定级"] = 0
            #公海中的a,b,c司机
            dfaa=dfa1[dfa1['重新定级'].isin([1])]
            dfab=dfa1[dfa1['重新定级'].isin([2])]
            dfac=dfa1[dfa1['重新定级'].isin([3])]
            '''



   

In [ ]:
#判断待补充小队的司机是否可以全部从公海提取
#if dfa2的行>=a，则可以补充dfa2中的a行进待补充小队
#if dfa2的行<a,则将dfa2剩余的所有行补充进待补充小队，从待剔除人数最多的小队中抽取a-已从公海补充的司机数补充进待补充小队。

#if dfa2的行>=b，则可以补充dfa2中的b行进待补充小队
#if dfa2的行<b,则将dfa2剩余的所有行补充进待补充小队，从待剔除人数最多的小队中抽取b-已从公海补充的司机数补充进待补充小队。

#if dfa2的行>=c，则可以补充dfa2中的c行进待补充小队
#if dfa2的行<c,则将dfa2剩余的所有行补充进待补充小队，从待剔除人数最多的小队中抽取c-已从公海补充的司机数补充进待补充小队。

#选择补充进待补充小队的司机
dfa5=dfa2[0:a]
dfa6=dfa3[0:b]
dfa7=dfa4[0:c]

#判断待剔除人数的小队队员的重新定级



#选好的司机补充进待补充小队

    